In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
import chebyshev_fitter, spline_fitter, sln_letter_fit
from sln_letter_fit import FitParams, OptimizationLoggingParams
import loader, plotting

%load_ext autoreload
%autoreload 1
%aimport chebyshev_fitter, spline_fitter, sln_letter_fit, loader, plotting

In [ ]:
# Util functions
def rms(err):
    return np.sqrt(np.mean(np.sum(np.square(err), axis=1), axis=0))
def error(strokes, sol):
    err = np.vstack(strokes)[:, 1:] - sol['txy_from_params'][:, 1:]
    return rms(err)
def error_letter(letter, sols):
    err = lambda strokes, sol: np.vstack(strokes)[:, 1:] - sol['txy_from_params'][:, 1:]
    errs = np.vstack([err(strokes, sol) for strokes, sol in zip(letter, sols)])
    return rms(errs)


# Fit trajectory

In [ ]:
poly_orders = list(range(2, 5))
fig, axes = plt.subplots(3, len(poly_orders), figsize=(20, 17))
strokes = loader.load_segments('D', index=1)

# Fit with Spline
for ax, poly_order in zip(axes[0], poly_orders):
    sol, _, _, _ = spline_fitter.fit_trajectory(strokes, p_order=poly_order)
    print('Spline (order {:}) RMSE: {:}'.format(poly_order, error(strokes, sol)))
    plotting.plot_trajectory(ax, strokes, sol)
    ax.plot(sol['params'][:, 1], sol['params'][:, 2], 'm*', label='Spline Nodes', markersize=15)
    ax.legend(loc='upper right')
    ax.set_title('Spline (order {:}) Fit, RMSE = {:.5e}'.format(poly_order, error(strokes, sol)))

# Fit with Chebyshev
for ax, poly_order in zip(axes[1], poly_orders):
    sol, _, _, stroke_indices = chebyshev_fitter.fit_trajectory(strokes, p_order=poly_order)
    print('Chebyshev (order {:}) RMSE: {:}'.format(poly_order, error(strokes, sol)))
    plotting.plot_trajectory(ax, strokes, sol)
    ax.set_title('Chebyshev (order {:}) Fit, RMSE = {:.5e}'.format(poly_order, error(strokes, sol)))

# Fit with SLN
sol, _ = sln_letter_fit.fit_and_plot_trajectory(axes[-1][-1],
                                                strokes,
                                                max_iters=50,
                                                log_history=False,
                                                pbar_description='Fitting SLN')
print('SLN RMSE: {:}'.format(error(strokes, sol)))
axes[-1][-1].set_title('SLN Fit, RMSE = {:.5e}'.format(error(strokes, sol)));

# Fit Letter

In [ ]:
poly_orders = list(range(2, 5))
iters = [50, 75, 100]
fig, axes = plt.subplots(3, len(poly_orders), figsize=(20, 12))
letter_ = 'B'
letter = loader.load_segments(letter_, index=None)

def process(sols, name):
    print(name + ' RMSE: {:}'.format(error_letter(letter, sols)))
    plotting.plot_letter(ax, letter, sols)
    ax.set_title(name + ' Fit, RMSE = {:.5e}'.format(error_letter(letter, sols)))

# Fit with Spline
for ax, poly_order in zip(axes[0], poly_orders):
    all_sols_and_histories = spline_fitter.fit_letter(letter, p_order=poly_order)
    process([sol for sol, _ in all_sols_and_histories], 'Spline (order {:})'.format(poly_order))
    for sol, _ in all_sols_and_histories:
        ax.plot(sol['params'][:, 1], sol['params'][:, 2], 'm*', label='Spline Nodes', markersize=10)

# Fit with Chebyshev
for ax, poly_order in zip(axes[1], poly_orders):
    all_sols_and_histories = chebyshev_fitter.fit_letter(letter, p_order=poly_order)
    process([sol for sol, _ in all_sols_and_histories], 'Chebyshev (order {:})'.format(poly_order))

# Fit with SLN
all_sols_and_histories = np.load('data/optimized_{:}.npy'.format(letter_), allow_pickle=True)
# all_sols_and_histories = sln_letter_fit.fit_letter(letter,
#                                                    max_iters=iters[-1],
#                                                    log_history=True,
#                                                    pbar_description_prefix='SLN Fitting Letter ' +
#                                                    letter_)
for ax, iter in zip(axes[2], iters):
    process([hist[iter] for _, hist in all_sols_and_histories], 'SLN ({:} iters)'.format(iter))

# Fit all letters

In [ ]:
# Processing / precomputing

poly_orders = list(range(2, 5))
iters = [100, 200, -1]
all_RMSE_errors = {}
all_data = {}

plt.ioff()
for letter_ in tqdm.tqdm('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    fig, axes = plt.subplots(3, len(poly_orders), figsize=(20, 12))
    letter = loader.load_segments(letter_, index=None)

    def process(sols, name, custom_info=None):
        plotting.plot_letter(ax, letter, sols)
        all_data[name.split(' ')[0], letter_, custom_info] = sols
        try:
            all_RMSE_errors[name.split(' ')[0], letter_, custom_info] = error_letter(letter, sols)
            # print('{:} - {:} RMSE: {:}'.format(letter_, name, all_RMSE_errors[letter_]))
            ax.set_title(name + ' Fit, RMSE = {:.5e}'.format(error_letter(letter, sols)))
        except ValueError:
            print('Calculation failed on letter {:}'.format(letter_))

    # Fit with Spline
    for ax, poly_order in zip(axes[0], poly_orders):
        all_sols_and_histories = spline_fitter.fit_letter(letter, p_order=poly_order)
        process([sol for sol, _ in all_sols_and_histories],
                'Spline (order {:})'.format(poly_order), poly_order)
        for sol, _ in all_sols_and_histories:
            ax.plot(sol['params'][:, 1],
                    sol['params'][:, 2],
                    'm*',
                    label='Spline Nodes',
                    markersize=10)

    # Fit with Chebyshev
    for ax, poly_order in zip(axes[1], poly_orders):
        all_sols_and_histories = chebyshev_fitter.fit_letter(letter, p_order=poly_order)
        process([sol for sol, _ in all_sols_and_histories],
                'Chebyshev (order {:})'.format(poly_order), poly_order)

    # Fit with SLN
    all_sols_and_histories = np.load('data/optimized_{:}.npy'.format(letter_), allow_pickle=True)
    for ax, iter in zip(axes[2], iters):
        process([hist[iter] for _, hist in all_sols_and_histories], 'SLN ({:} iters)'.format(iter),
                iter)

    fig.savefig('data/compare_spline_cheby_sln_{:}.svg'.format(letter_))
    plt.close(fig)


In [ ]:
# Displaying

def filt(name, special):
    return [v for k, v in all_RMSE_errors.items() if k[0] == name and k[2] == special]

bins = np.linspace(0, 0.1, 50)

fig, axes = plt.subplots(3, 3, figsize=(15, 7), sharex=True, sharey=True)
fig.tight_layout()
for i, order in enumerate(poly_orders):
    axes[0][i].hist(filt('Spline', order), bins=bins)
    axes[0][i].set_title('RMSEs for Spline Fits of Order {:}'.format(order))
for i, order in enumerate(poly_orders):
    axes[1][i].hist(filt('Chebyshev', order), bins=bins)
    axes[1][i].set_title('RMSEs for Chebyshev Fits of Order {:}'.format(order))
for i, iter in enumerate(iters):
    axes[2][i].hist(filt('SLN', iter), bins=bins)
    axes[2][i].set_title('RMSEs for SLN Fits after {:} iters'.format(iter))
for ax in axes.flatten():
    ax.set_xlim(0, bins.max())

display(fig)